# 1. Import packages

In [1]:
import sys
sys.path.append('..')
from modules import preprocess, benchmarking, graphics

Invoking __init__.py for modules


# 2. Load dataset

In [2]:
df = preprocess.load_csv_data('../dataset/Occupancy_Estimation.csv')

# 3. Data preprocessing
- Step 1: Transform features
  - Binary results for Room_Occupancy_Count
  - Drop Date and Time features
- Step 2: Clean data
  - Drop duplicated rows

In [3]:
# Step 1: Recode Room_Occupancy_Count column
preprocess.recode_dataset_output(df)

# Step 2: Filter Date and Time columns
preprocess.remove_time_columns(df)

df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


# 4. Benchmarking

In [4]:
# Split dataset into features and output
X = preprocess.get_features(df)
y = preprocess.get_output(df)

# Split dataset into training and test
X_train, X_test, y_train, y_test = preprocess.split_dataset(X, y)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


## Create benchmarking Dataframe:
- Create benchmarking's df structure.
- Track models training with CodeCarbon and Eco2AI.
- Store in benchmarking's df tracking results.
- Store in benchmarking's df evaluation metrics.

In [ ]:
df_benchmarking = benchmarking.create_benchmarking(X_train, y_train, X_test, y_test)

[codecarbon INFO @ 19:05:12] [setup] RAM Tracking...
[codecarbon INFO @ 19:05:12] [setup] GPU Tracking...
[codecarbon INFO @ 19:05:12] No GPU found.
[codecarbon INFO @ 19:05:12] [setup] CPU Tracking...
[codecarbon WARNING @ 19:05:12] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 19:05:14] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:05:14] >>> Tracker's metadata:
[codecarbon INFO @ 19:05:14]   Platform system: Linux-5.19.0-43-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 19:05:14]   Python version: 3.10.8
[codecarbon INFO @ 19:05:14]   Available RAM : 4.946 GB
[codecarbon INFO @ 19:05:14]   CPU count: 4
[codecarbon INFO @ 19:05:14]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:05:14]   GPU count: None
[codecarbon INFO @ 19:05:14]   GPU model: None


Fitting 5 folds for each of 120 candidates, totalling 600 fits


[codecarbon INFO @ 19:05:32] Energy consumed for RAM : 0.000000 kWh. RAM Power : 0.0828866958618164 W
[codecarbon INFO @ 19:05:32] Energy consumed for all CPUs : 0.000136 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:05:32] 0.000136 kWh of electricity used since the begining.
[codecarbon INFO @ 19:05:47] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.08304834365844727 W
[codecarbon INFO @ 19:05:47] Energy consumed for all CPUs : 0.000271 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:05:47] 0.000272 kWh of electricity used since the begining.
[codecarbon INFO @ 19:05:51] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.08312129974365234 W
[codecarbon INFO @ 19:05:51] Energy consumed for all CPUs : 0.000306 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:05:51] 0.000307 kWh of electricity used since the begining.
[codecarbon INFO @ 19:05:51] [setup] RAM Tracking...
[codecarbon INFO @ 19:05:51] [setup] GPU Tracking...
[codecarbon INFO @ 19:05:51] No GPU found.
[co

Best estimator LR:  Pipeline(steps=[('scaler', RobustScaler()),
                ('estimator',
                 LogisticRegression(C=78.47599703514607, max_iter=400,
                                    penalty='l1', solver='liblinear'))])
Precision:  1.0
Recall:  0.9982993197278912
F1:  0.9991489361702128


[codecarbon INFO @ 19:05:53] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:05:53] >>> Tracker's metadata:
[codecarbon INFO @ 19:05:53]   Platform system: Linux-5.19.0-43-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 19:05:53]   Python version: 3.10.8
[codecarbon INFO @ 19:05:53]   Available RAM : 4.946 GB
[codecarbon INFO @ 19:05:53]   CPU count: 4
[codecarbon INFO @ 19:05:53]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:05:53]   GPU count: None
[codecarbon INFO @ 19:05:53]   GPU model: None


...

[codecarbon INFO @ 19:06:58] Energy consumed for RAM : 0.000000 kWh. RAM Power : 0.08996772766113283 W
[codecarbon INFO @ 19:06:58] Energy consumed for all CPUs : 0.000136 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:06:58] 0.000136 kWh of electricity used since the begining.
[codecarbon INFO @ 19:07:13] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.08989906311035156 W
[codecarbon INFO @ 19:07:13] Energy consumed for all CPUs : 0.000271 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:07:13] 0.000272 kWh of electricity used since the begining.
[codecarbon INFO @ 19:07:28] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.08406114578247072 W
[codecarbon INFO @ 19:07:28] Energy consumed for all CPUs : 0.000406 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:07:28] 0.000408 kWh of electricity used since the begining.
[codecarbon INFO @ 19:07:43] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.08445167541503906 W
[codecarbon INFO @ 19:07:43] Energy consumed for

Best estimator SVM:  SVC(C=1, gamma=0.001)
Precision:  1.0
Recall:  0.9982993197278912
F1:  0.9991489361702128


[codecarbon INFO @ 19:07:59] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:07:59] >>> Tracker's metadata:
[codecarbon INFO @ 19:07:59]   Platform system: Linux-5.19.0-43-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 19:07:59]   Python version: 3.10.8
[codecarbon INFO @ 19:07:59]   Available RAM : 4.946 GB
[codecarbon INFO @ 19:07:59]   CPU count: 4
[codecarbon INFO @ 19:07:59]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:07:59]   GPU count: None
[codecarbon INFO @ 19:07:59]   GPU model: None


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[codecarbon INFO @ 19:08:17] Energy consumed for RAM : 0.000000 kWh. RAM Power : 0.08443880081176759 W
[codecarbon INFO @ 19:08:17] Energy consumed for all CPUs : 0.000139 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:08:17] 0.000139 kWh of electricity used since the begining.
[codecarbon INFO @ 19:08:32] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.08509969711303711 W
[codecarbon INFO @ 19:08:32] Energy consumed for all CPUs : 0.000273 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:08:32] 0.000274 kWh of electricity used since the begining.
[codecarbon INFO @ 19:08:47] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.08557462692260742 W
[codecarbon INFO @ 19:08:47] Energy consumed for all CPUs : 0.000409 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:08:47] 0.000410 kWh of electricity used since the begining.
[codecarbon INFO @ 19:09:02] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.0855860710144043 W
[codecarbon INFO @ 19:09:02] Energy consumed for 

In [ ]:
df_benchmarking

## Save benchmarking's DataFrame in csv file

In [ ]:
preprocess.save_in_csv_file(df_benchmarking, 'benchmarking.csv')

## Graphics

In [ ]:
graphics.plot_models_energy_consumed(df_benchmarking)

In [ ]:
graphics.plot_models_evaluation_metrics(df_benchmarking)